<i>Copyright (c) Recommenders contributors.</i>

<i>Licensed under the MIT License.</i>

# Bayesian Personalized Ranking (BPR)

This notebook serves as an introduction to Bayesian Personalized Ranking (BPR) model for implicit feedback.  In this tutorial, we focus on learning the BPR model using matrix factorization approach, hence, the model is sometimes also named BPRMF.

The implementation of the model is from [Cornac](https://github.com/PreferredAI/cornac), which is a framework for recommender systems with a focus on models leveraging auxiliary data (e.g., item descriptive text and image, social network, etc).

## 0 Global Settings and Imports

In [24]:
import os
import sys
import cornac
import pandas as pd
import dataset_loader

from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import map, ndcg_at_k, mae, rmse, novelty, historical_item_novelty, user_item_serendipity, user_serendipity, serendipity, catalog_coverage, distributional_coverage
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED
from metrics import precision_at_k, recall_at_k, mrr, accuracy, user_coverage, item_coverage, personalization_score, intra_list_similarity_score, intra_list_dissimilarity

print(f"System version: {sys.version}")
print(f"Cornac version: {cornac.__version__}")

System version: 3.9.21 (main, Dec 11 2024, 16:35:24) [MSC v.1929 64 bit (AMD64)]
Cornac version: 2.3.0


In [25]:
# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

# top k items to recommend
TOP_K = 10

# Model parameters
NUM_FACTORS = 200
NUM_EPOCHS = 100

## 1 BPR Algorithm

### 1.1 Personalized Ranking from Implicit Feedback

The task of personalized ranking aims at providing each user a ranked list of items (recommendations). This is very common in scenarios where recommender systems are based on implicit user behavior (e.g. purchases, clicks). The available observations are only positive feedback where the non-observed ones are a mixture of real negative feedback and missing values.

One usual approach for item recommendation is directly predicting a preference score $\hat{x}_{u,i}$ given to item $i$ by user $u$. This method uses the **pointwise loss** to optimize the model. Methods based on pointwise learning usually follow a regression framework by minimizing the  squared loss between $\hat{x}_{u,i}$ its target value ${x}_{u,i}$. Unfortunately, this method doesn't work well with ranking.

<p align="center">
<img src="https://recodatasets.z20.web.core.windows.net/images/bpr1.png" width="400px">
</p>

In the implicit feedback scenario the observed data is positive feedback. As it is shown in the figure above, the observed data (image to the left) is known, but the unobserved data is unknown. The negative data is usually generated by filling the matrix with 0 (image to the right). 

Using the pointwise loss, the model is trained to predict as 1 the elements in the observed dataset and 0 for the rest. The problem with this approach is that all elements the model should rank in the future are presented as negative feedback during training but in reality, some of them are unknown items and some of them are true negatives. 

BPR uses a different approach by using item pairs $(i, j)$ and optimizing for the correct ranking given preference of user $u$, thus, there are notions of *positive* and *negative* items. The training data $D_S : U \times I \times I$ is defined as:

$$D_S = \{(u, i, j) \mid i \in I^{+}_{u} \wedge j \in I \setminus I^{+}_{u}\}$$

where user $u$ is assumed to prefer $i$ over $j$ (i.e. $i$ is a *positive item* and $j$ is a *negative item*).

This approach uses the **pairwise loss**. The intuition behind it is that observed entries should be ranked higher than the unobserved ones. Then instead of minimizing the loss between $\hat{x}_{u,i}$ and ${x}_{u,i}$, pairwise learning maximizes the margin between the observed entry $\hat{x}_{u,i}$ and the unobserved entry $\hat{x}_{u,j}$.

<p align="center">
<img src="https://recodatasets.z20.web.core.windows.net/images/bpr2.png" width="400px">
</p>

In the figure above, pairwise learning is explained. For each user, a pairwise matrix of items is generated. The plus indicates that the user prefers item *i* over item *j*, and the minus is that the user prefers item *j* over *i*. The interrogation indicates that we don't know the user's preference.

The pairwise setup has several advantages:
* The training set is defined as positive, negative and unknown preferences. 
* The missing values between two non-observed items are exactly the item pairs that have to be ranked by the model.
* The training data is created for the actual objective of ranking.
* Pairwise ranking is not restricted to implicit feedback and can be used in explicit feedback by setting user preferences as pairs (i.e. user *u* prefers item *a* more than item *b*).

### 1.2 Pairwise Objective Function

From the Bayesian perspective, BPR maximizes the posterior probability over the model parameters $\Theta$ by optimizing the likelihood function $p(i >_{u} j | \Theta)$ and the prior probability $p(\Theta)$.

$$p(\Theta \mid >_{u}) \propto p(i >_{u} j \mid \Theta) \times p(\Theta)$$

The joint probability of the likelihood over all users $u \in U$ can be simplified to:

$$ \prod_{u \in U} p(>_{u} \mid \Theta) = \prod_{(u, i, j) \in D_S} p(i >_{u} j \mid \Theta) $$

The individual probability that a user $u$ prefers item $i$ to item $j$ can be defined as:

$$ p(i >_{u} j \mid \Theta) = \sigma (\hat{x}_{uij}(\Theta)) $$

where $\sigma$ is the logistic sigmoid:

$$ \sigma(x) = \frac{1}{1 + e^{-x}} $$

The preference scoring function $\hat{x}_{uij}(\Theta)$ could be an arbitrary real-valued function of the model parameter $\Theta$.  Thus, it makes BPR a general framework for modeling the relationship between triplets $(u, i, j)$ where different model classes like matrix factorization could be used for estimating $\hat{x}_{uij}(\Theta)$.

For the prior, one of the common pratices is to choose $p(\Theta)$ following a normal distribution, which results in a nice form of L2 regularization in the final log-form of the objective function.

$$ p(\Theta) \sim N(0, \Sigma_{\Theta}) $$

To reduce the complexity of the model, all parameters $\Theta$ are assumed to be independent and having the same variance, which gives a simpler form of the co-variance matrix $\Sigma_{\Theta} = \lambda_{\Theta}I$.  Thus, there are less number of hyperparameters to be determined.

The final objective of the maximum posterior estimator:

$$ J = \sum_{(u, i, j) \in D_S} \text{ln } \sigma(\hat{x}_{uij}) - \lambda_{\Theta} ||\Theta||^2 $$

where $\lambda_\Theta$ are the model specific regularization parameters.


### 1.3 Learning with Matrix Factorization

#### Stochastic Gradient Descent

As the defined objective function is differentiable, gradient descent based method for optimization is naturally adopted.  The gradient of the objective $J$ with respect to the model parameters:

$$
\begin{align}
\frac{\partial J}{\partial \Theta} & = \sum_{(u, i, j) \in D_S} \frac{\partial}{\partial \Theta} \text{ln} \ \sigma(\hat{x}_{uij}) - \lambda_{\Theta} \frac{\partial}{\partial \Theta} ||\Theta||^2 \\
& \propto \sum_{(u, i, j) \in D_S} \frac{-e^{-\hat{x}_{uij}}}{1 + e^{-\hat{x}_{uij}}} \cdot  \frac{\partial}{\partial \Theta} \hat{x}_{uij} - \lambda_{\Theta} \Theta
\end{align}
$$

Due to slow convergence of full gradient descent, we prefer using stochastic gradient descent to optimize the BPR model.  For each triplet $(u, i, j) \in D_S$, the update rule for the parameters:

$$ \Theta \leftarrow \Theta + \alpha \Big( \frac{e^{-\hat{x}_{uij}}}{1 + e^{-\hat{x}_{uij}}} \cdot \frac{\partial}{\partial \Theta} \hat{x}_{uij} + \lambda_\Theta \Theta \Big) $$

#### Matrix Factorization for Preference Approximation

As mentioned earlier, the preference scoring function $\hat{x}_{uij}(\Theta)$ could be approximated by any real-valued function.  First, the estimator $\hat{x}_{uij}$ is decomposed into:

$$ \hat{x}_{uij} = \hat{x}_{ui} - \hat{x}_{uj} $$

The problem of estimating $\hat{x}_{ui}$ is a standard collaborative filtering formulation, where matrix factorization approach has shown to be very effective.  The prediction formula can written as dot product between user feature vector $w_u$ and item feature vector $h_i$:

$$ \hat{x}_{ui} = \langle w_u , h_i \rangle = \sum_{f=1}^{k} w_{uf} \cdot h_{if} $$

The  derivatives of matrix factorization with respect to the model parameters are:

$$
\frac{\partial}{\partial \theta} \hat{x}_{uij} = 
\begin{cases}
    (h_{if} - h_{jf})  & \text{if } \theta = w_{uf} \\
    w_{uf}             & \text{if } \theta = h_{if} \\
    -w_{uf}            & \text{if } \theta = h_{jf} \\
    0                  & \text{else}
\end{cases}
$$

In theory, any kernel can be used to estimate $\hat{x}_{ui}$ besides the dot product $ \langle \cdot , \cdot \rangle $.  For example, k-Nearest-Neighbor (kNN) has also been shown to achieve good performance.

#### Analogies to AUC optimization

By optimizing the objective function of BPR model, we effectively maximizing [AUC](https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5) measurement. To keep the notebook focused, please refer to the [paper](https://arxiv.org/ftp/arxiv/papers/1205/1205.2618.pdf) for details of the analysis (Section 4.1.1).

## 2 Cornac implementation of BPR

BPR is implemented in the [Cornac](https://cornac.readthedocs.io/en/latest/index.html) framework as part of the model collections.
* Detailed documentations of the BPR model in Cornac can be found [here](https://cornac.readthedocs.io/en/latest/models.html#bayesian-personalized-ranking-bpr).
* Source codes of the BPR implementation is available on the Cornac Github repository, which can be found [here](https://github.com/PreferredAI/cornac/blob/master/cornac/models/bpr/recom_bpr.pyx).


## 3 Cornac BPR movie recommender


### 3.1 Load and split data

To evaluate the performance of item recommendation, we adopted the provided `python_random_split` tool for the consistency.  Data is randomly split into training and test sets with the ratio of 75/25.


Note that Cornac also cover different [built-in schemes](https://cornac.readthedocs.io/en/latest/eval_methods.html) for model evaluation.

In [26]:
data = dataset_loader.loader("movielens", want_col=["userID", "itemID", "rating", "timestamp", 'title', 'genres'], num_rows=5000)

data.head()

,userID,itemID,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [27]:
train, test = python_random_split(data, 0.75)

### 3.2 Cornac Dataset

To work with models implemented in Cornac, we need to construct an object from [Dataset](https://cornac.readthedocs.io/en/latest/data.html#module-cornac.data.dataset) class.

Dataset Class in Cornac serves as the main object that the models will interact with.  In addition to data transformations, Dataset provides a bunch of useful iterators for looping through the data, as well as supporting different negative sampling techniques.

In [28]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 50
Number of items: 1787


### 3.3 Train the BPR model

The BPR has a few important parameters that we need to consider:

- `k`: controls the dimension of the latent space (i.e. the size of the vectors  $w_u$  and  $h_i$ ).
- `max_iter`: defines the number of iterations of the SGD procedure.
- `learning_rate`: controls the step size $\alpha$ in the gradient update rules.
- `lambda_reg`: controls the L2-Regularization $\lambda$ in the objective function.

Note that different values of `k` and `max_iter` will affect the training time.

We will here set `k` to 200, `max_iter` to 100, `learning_rate` to 0.01, and `lambda_reg` to 0.001. To train the model, we simply need to call the `fit()` method.

In [29]:
bpr = cornac.models.BPR(
    k=NUM_FACTORS,
    max_iter=NUM_EPOCHS,
    learning_rate=0.01,
    lambda_reg=0.001,
    verbose=True,
    seed=SEED
)

In [30]:
with Timer() as t:
    bpr.fit(train_set)
print("Took {} seconds for training.".format(t))

100%|██████████| 100/100 [00:00<00:00, 283.01it/s, correct=91.06%, skipped=9.28%]

Optimization finished!
Took 0.3603 seconds for training.


### 3.4 Prediction and Evaluation

Now that our model is trained, we can produce the ranked lists for recommendation.  Every recommender models in Cornac provide `rate()` and `rank()` methods for predicting item rated value as well as item ranked list for a given user.  To make use of the current evaluation schemes, we will through `predict()` and `predict_ranking()` functions inside `cornac_utils` to produce the predictions.

Note that BPR model is effectively designed for item ranking.  Hence, we only measure the performance using ranking metrics.

In [31]:
with Timer() as t:
    all_predictions = predict_ranking(bpr, train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))

Took 0.1122 seconds for prediction.


In [32]:
all_predictions.head()

,userID,itemID,prediction
0,1,1,1.780101
2,1,3,0.845180
3,1,4,-0.124073
4,1,5,0.354452
5,1,6,1.560619


In [33]:
all_predictions[all_predictions["userID"] == 1]

,userID,itemID,prediction
0,1,1,1.780101
2,1,3,0.845180
3,1,4,-0.124073
4,1,5,0.354452
5,1,6,1.560619
...,...,...,...
1782,1,98809,-0.437156
1783,1,102125,-0.341667
1784,1,106489,-0.324994
1785,1,106696,-0.377117


In [34]:
all_predictions[all_predictions["userID"] == 1].sort_values(by="prediction", ascending=False)

,userID,itemID,prediction
241,1,480,2.432905
290,1,593,2.295903
260,1,527,2.196138
183,1,356,2.046748
292,1,595,2.017572
...,...,...,...
579,1,1611,-0.718604
563,1,1539,-0.723286
1221,1,4473,-0.724165
982,1,3249,-0.748852


In [35]:
rows, columns = test.shape
rows

1250

In [36]:
merged_df = test.merge(all_predictions, on=["userID", "itemID"], how="inner")[["userID", "itemID", "rating", "prediction"]]
merged_df

,userID,itemID,rating,prediction
0,13,2,3.0,0.674165
1,24,339,3.0,0.577513
2,24,1120,4.0,-0.157839
3,7,2858,3.0,1.349550
4,23,1060,5.0,1.144544
...,...,...,...,...
909,3,2140,4.0,-0.017767
910,45,3977,2.0,-0.257457
911,22,6,5.0,2.035404
912,26,39,4.0,1.157254


In [37]:
user_counts = test["userID"].value_counts().reset_index()
user_counts.columns = ["userID", "count"]
user_counts

,userID,count
0,11,124
1,24,120
2,7,76
3,31,74
4,14,58
5,1,48
6,3,48
7,35,41
8,21,40
9,29,37


In [38]:
max_predictions = all_predictions.loc[all_predictions.groupby("userID")["prediction"].idxmax(), ["userID", "itemID", "prediction"]]
max_predictions

,userID,itemID,prediction
241,1,480,2.432905
1939,2,296,2.696571
3726,3,296,2.636324
5513,4,296,2.702422
7300,5,296,2.718505
9087,6,296,2.693036
10874,7,296,2.414536
12799,8,593,2.365722
14448,9,296,2.723147
16235,10,296,2.691511


In [39]:
# k = 1

# eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=k)
# eval_precision_k = precision_at_k(test, all_predictions, col_prediction='prediction', k=10)
# eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=k)
# eval_recall_k = recall_at_k(test, all_predictions, col_prediction='prediction', k=10)
# # eval_f1
# eval_mae = mae(test, all_predictions, col_prediction='prediction')
# eval_rmse = rmse(test, all_predictions, col_prediction='prediction')
# # eval_mrr
# eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=k)
# # eval_intra_list_diversity
# # eval_user_coverage
# # eval_item_coverage
# # eval_personalization

# # eval_novelty = novelty(train, max_predictions)
# # eval_historical_item_novelty = historical_item_novelty(train, max_predictions)
# # eval_user_item_serendipity = user_item_serendipity(train, max_predictions)
# # eval_user_serendipity = user_serendipity(train, max_predictions)
# # eval_serendipity = serendipity(train, max_predictions)
# # eval_catalog_coverage = catalog_coverage(train, max_predictions)
# # eval_distributional_coverage = distributional_coverage(train, max_predictions)

In [40]:
# print("Precision:\t%f" % eval_precision,
#       "Precision@K:\t%f" % eval_precision_k,
#       "Recall:\t%f" % eval_recall,
#       "Recall@K:\t%f" % eval_recall_k,
#       "MAE:\t%f" % eval_mae,
#       "RMSE:\t%f" % eval_rmse,
#       "NDCG:\t%f" % eval_ndcg,
#       # "Novelty:\t%f" % eval_novelty,
#       # "Serendipity:\t%f" % eval_serendipity,
#       # "Catalog coverage:\t%f" % eval_catalog_coverage,
#       # "Distributional coverage:\t%f" % eval_distributional_coverage,
#       sep='\n')

In [41]:
intra_list_similarity_score(data, all_predictions, feature_cols=['genres'])

0.2562684552087251

In [42]:
intra_list_dissimilarity(data, all_predictions, feature_cols=['genres'])

0.7437315447912749

In [43]:
personalization_score(train, max_predictions)

0.716734693877551

In [44]:
# eval_historical_item_novelty

In [45]:
# eval_user_item_serendipity

In [46]:
# eval_user_serendipity

## References

1. Rendle, S., Freudenthaler, C., Gantner, Z., & Schmidt-Thieme, L. (2009, June). BPR: Bayesian personalized ranking from implicit feedback. https://arxiv.org/ftp/arxiv/papers/1205/1205.2618.pdf
2. Pan, R., Zhou, Y., Cao, B., Liu, N. N., Lukose, R., Scholz, M., & Yang, Q. (2008, December). One-class collaborative filtering. https://cseweb.ucsd.edu/classes/fa17/cse291-b/reading/04781145.pdf
3. **Cornac** - A Comparative Framework for Multimodal Recommender Systems. https://cornac.preferred.ai/